In [19]:
# ==============================================================================
# 单元格 1: 环境设置、数据加载与数据字典创建
# Cell 1: Environment Setup, Data Loading & Data Dictionary Creation
# ==============================================================================

import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. 环境美化与设置 ---
# --- 1. Beautification and Settings ---
# 设置 Seaborn 样式，让图表更美观
# Set the Seaborn style for more aesthetically pleasing plots.
sns.set_style("whitegrid")
# 设置 Matplotlib 支持中文显示 (如果需要)
# Configure Matplotlib to support Chinese characters (if needed).
plt.rcParams['font.sans-serif'] = ['Heiti TC'] # 'Heiti TC' 是一种常用的Mac中文字体
plt.rcParams['axes.unicode_minus'] = False # 解决负号显示问题

# --- 2. 定义文件路径 ---
# --- 2. Define File Paths ---
# 定义原始数据所在的路径
# Define the path to the raw data directory.
raw_data_path = '../data_acquisition/raw_data/'

# --- 3. 创建数据字典 ---
# --- 3. Create Data Dictionary ---
# 加载我们之前分析过的“密码本”文件
# Load the 'DataProperties.json' file, which acts as our data's "codebook".
data_properties_file = os.path.join(raw_data_path, 'DataProperties.json')

# 初始化一个空的DataFrame来存储数据字典
# Initialize an empty DataFrame to store the data dictionary.
df_properties = pd.DataFrame()

try:
    # 读取JSON文件
    # Read the JSON file into a DataFrame.
    df_properties = pd.read_json(data_properties_file)
    
    # 我们只关心类型为 'Topic' 的行，因为它们代表了实际的数据列
    # We are only interested in rows of type 'Topic' as they represent the actual data columns.
    df_properties = df_properties[df_properties['Type'] == 'Topic'].copy()

    # 选取我们最关心的几列，并重命名以提高可读性
    # Select the columns we care most about and rename them for better readability.
    df_properties = df_properties[['Key', 'Title', 'Description', 'Unit']]
    df_properties.rename(columns={
        'Key': '指标技术名 (Technical Name)',
        'Title': '指标可读名 (Readable Name)',
        'Description': '描述 (Description)',
        'Unit': '单位 (Unit)'
    }, inplace=True)

    # 将指标技术名设置成索引，方便快速查找
    # Set the technical name as the index for quick lookups.
    df_properties.set_index('指标技术名 (Technical Name)', inplace=True)
    
    print("--- 数据字典创建成功 ---")
    print("Data Dictionary created successfully.")
    
except FileNotFoundError:
    print(f"错误：未找到数据字典文件：{data_properties_file}")
    print(f"Error: Data dictionary file not found at: {data_properties_file}")
except Exception as e:
    print(f"创建数据字典时发生错误：{e}")
    print(f"An error occurred while creating the data dictionary: {e}")

# 显示数据字典的前10行，让我们看看它的样子
# Display the first 10 rows of the data dictionary to see what it looks like.
df_properties.head(10)


--- 数据字典创建成功 ---
Data Dictionary created successfully.


,指标可读名 (Readable Name),描述 (Description),单位 (Unit)
指标技术名 (Technical Name),,,
CompensationOfEmployees_1,Compensation of employees,Compensation of employees is the total remuner...,million euros
WagesAndSalaries_2,Wages and salaries,Wages and salaries include income taxes and em...,million euros
EmployersSocialContributions_3,Employers' social contributions,Employers' social contributions consist of pay...,million euros
WageCosts_4,Wage costs,"The total of wages, social contributions paid ...",million euros
CompensationOfEmployees_5,Compensation of employees,Compensation of employees is the total remuner...,%
WagesAndSalaries_6,Wages and salaries,Wages and salaries include income taxes and em...,%
WageCosts_7,Wage costs,"The total of wages, social contributions paid ...",%
CompensationPerFte_8,Compensation per fte,Compensation of employees per full-time equiva...,"1,000 euro"
WagesPerFte_9,Wages per fte,Wages and salaries per full-time equivalent (f...,"1,000 euro"


In [20]:
# ==============================================================================
# 单元格 2: 合并数据以创建主数据表 (df_master)
# Cell 2: Merging Data to Create the Master DataFrame (df_master)
# ==============================================================================

# --- 1. 加载所有需要用于合并的数据文件 ---
# --- 1. Load all data files required for merging ---

# 定义需要加载的文件列表
# Define the list of files to load.
files_to_load = ['TypedDataSet.json', 'Periods.json', 'SectorBranchesSIC2008.json']

# 创建一个字典来存储所有加载的数据框
# Create a dictionary to store all loaded DataFrames.
dfs = {}

print("--- 开始加载数据文件 ---")
print("--- Starting to load data files ---")
for file_name in files_to_load:
    file_path = os.path.join(raw_data_path, file_name)
    df_name = file_name.replace('.json', '')
    try:
        # CBS的数据通常有一个'value'键，其中包含真正的数据列表
        # CBS data often has a 'value' key containing the actual list of data.
        raw_df = pd.read_json(file_path)
        if 'value' in raw_df.columns:
            dfs[df_name] = pd.DataFrame(list(raw_df['value']))
        else:
            # 如果没有'value'键，则直接使用读取的DataFrame
            # If there is no 'value' key, use the read DataFrame directly.
            dfs[df_name] = raw_df
        print(f"成功加载 '{file_name}'")
        print(f"Successfully loaded '{file_name}'")
    except Exception as e:
        print(f"加载 '{file_name}' 时出错: {e}")
        print(f"Error loading '{file_name}': {e}")
print("--------------------------\n")


# --- 2. 合并数据表 ---
# --- 2. Merge DataFrames ---

# 初始化主数据表为核心数据集
# Initialize the master DataFrame with the core dataset.
df_master = dfs.get('TypedDataSet', pd.DataFrame())

if not df_master.empty:
    # 第1次合并：加入时间信息 (Periods)
    # 1st Merge: Join with time information (Periods).
    print("--- 正在合并时间数据... ---")
    print("--- Merging with Periods data... ---")
    df_master = pd.merge(
        df_master,
        dfs.get('Periods', pd.DataFrame()),
        left_on='Periods',
        right_on='Key',
        how='left'
    )
    # 重命名列并删除无用的Key列
    # Rename the column and drop the now-useless Key column.
    df_master.rename(columns={'Title': 'Year'}, inplace=True)
    df_master.drop('Key', axis=1, inplace=True)
    print("时间数据合并完成。")
    print("Periods data merged.\n")

    # 第2次合并：加入行业信息 (Sector/Branches)
    # 2nd Merge: Join with industry information (Sector/Branches).
    print("--- 正在合并行业数据... ---")
    print("--- Merging with Sector/Branch data... ---")
    df_master = pd.merge(
        df_master,
        dfs.get('SectorBranchesSIC2008', pd.DataFrame()),
        left_on='SectorBranchesSIC2008',
        right_on='Key',
        how='left'
    )
    # 重命名列并删除无用的Key列
    # Rename the column and drop the now-useless Key column.
    df_master.rename(columns={'Title': 'Sector'}, inplace=True)
    df_master.drop('Key', axis=1, inplace=True)
    print("行业数据合并完成。")
    print("Sector/Branch data merged.\n")

    # --- 3. 清理与整理主数据表 ---
    # --- 3. Tidy up the Master DataFrame ---
    print("--- 正在整理主数据表... ---")
    print("--- Tidying up the master DataFrame... ---")

    # 删除现在多余的、用于合并的原始ID列
    # Drop the original ID columns used for merging, which are now redundant.
    columns_to_drop = ['Periods', 'SectorBranchesSIC2008']
    df_master.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    # 调整列顺序，将 'Year' 和 'Sector' 移动到最前面，方便观察
    # Rearrange columns to move 'Year' and 'Sector' to the front for easier observation.
    cols = df_master.columns.tolist()
    # 找到 'Year' 和 'Sector' 并将它们从列表中移除
    # Find 'Year' and 'Sector' and remove them from the list.
    if 'Year' in cols:
        year_col = cols.pop(cols.index('Year'))
        cols.insert(0, year_col) # 插入到最前面
    if 'Sector' in cols:
        sector_col = cols.pop(cols.index('Sector'))
        cols.insert(1, sector_col) # 插入到第二个位置
        
    df_master = df_master[cols]
    
    print("主数据表整理完毕。")
    print("Master DataFrame tidied up.\n")

    # --- 4. 最终验证 ---
    # --- 4. Final Verification ---
    print("--- 主数据表信息 (Master DataFrame Info) ---")
    df_master.info()
    print("\n--- 主数据表前5行 (Master DataFrame Head) ---")
    print(df_master.head())
else:
    print("核心数据集 'TypedDataSet.json' 未能加载，合并中止。")
    print("Core dataset 'TypedDataSet.json' failed to load, merging aborted.")


--- 开始加载数据文件 ---
--- Starting to load data files ---
成功加载 'TypedDataSet.json'
Successfully loaded 'TypedDataSet.json'
成功加载 'Periods.json'
Successfully loaded 'Periods.json'
成功加载 'SectorBranchesSIC2008.json'
Successfully loaded 'SectorBranchesSIC2008.json'
--------------------------

--- 正在合并时间数据... ---
--- Merging with Periods data... ---
时间数据合并完成。
Periods data merged.

--- 正在合并行业数据... ---
--- Merging with Sector/Branch data... ---
行业数据合并完成。
Sector/Branch data merged.

--- 正在整理主数据表... ---
--- Tidying up the master DataFrame... ---
主数据表整理完毕。
Master DataFrame tidied up.

--- 主数据表信息 (Master DataFrame Info) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 43 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Year                            3000 non-null   int64  
 1   Sector                          3000 non-null   object 
 2   ID                           

In [21]:
# ==============================================================================
# SPRINT 1 - MILESTONE 3: Interactive Visualization with Pyecharts
# ==============================================================================
#
# Task 1: Environment Setup
# --------------------------
#
# First, we need to install the pyecharts library.
# We run this command directly in the notebook to ensure the environment is always ready.
# The '!' symbol allows us to execute a shell command from a notebook cell.
# The '-q' flag is for a "quiet" installation, reducing the amount of log output.

%pip install -q pyecharts



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
# Task 2: Interactive Visualization of Overall Trend
# --------------------------------------------------
#
# Let's begin our analysis by visualizing the average salary trend for all industries.

from pyecharts import options as opts
from pyecharts.charts import Line

# 1. Prepare the data: Group by 'Year' and calculate the mean of 'WagesPerFte_9'
avg_salary_by_year = df_analysis.groupby('Year')['WagesPerFte_9'].mean().round(0).astype(int)

# 2. Create the chart object
line_chart = (
    Line()
    .add_xaxis(xaxis_data=avg_salary_by_year.index.astype(str).tolist())
    .add_yaxis(
        series_name="Average Annual Salary (FTE)",
        y_axis=avg_salary_by_year.values.tolist(),
        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max"), opts.MarkPointItem(type_="min")]),
        markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")])
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Average Salary Trend (All Industries)", subtitle="Source: Dutch CBS"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        yaxis_opts=opts.AxisOpts(name="Salary (€)"),
        datazoom_opts=opts.DataZoomOpts(is_show=True, range_start=0, range_end=100),
    )
)

# 3. Render the chart in the notebook
line_chart.render_notebook()


In [23]:
# ==============================================================================
# SPRINT 1 - MILESTONE 4: Calculating the "Big Numbers"
# ==============================================================================
#
# Task 3.1: Data Preparation for Growth/Recession Analysis
# --------------------------------------------------------

import pandas as pd
from pyecharts.charts import Bar

# 1. Determine the start and end years
start_year = df_analysis['Year'].min()
end_year = df_analysis['Year'].max()

print(f"Analyzing growth from {start_year} to {end_year}")

# 2. Filter data for these years, excluding the 'All economic activities' aggregate
df_start = df_analysis[(df_analysis['Year'] == start_year) & (df_analysis['Sector'] != 'A-U All economic activities')]
df_end = df_analysis[(df_analysis['Year'] == end_year) & (df_analysis['Sector'] != 'A-U All economic activities')]

# 3. Merge to get start and end salaries side-by-side
df_growth = pd.merge(
    df_start[['Sector', 'WagesPerFte_9']],
    df_end[['Sector', 'WagesPerFte_9']],
    on='Sector',
    suffixes=('_start', '_end')
)

# 4. Calculate the growth multiple
df_growth['GrowthMultiple'] = (df_growth['WagesPerFte_9_end'] / df_growth['WagesPerFte_9_start']).round(2)

df_growth.head()


Analyzing growth from 1995 to 2024


,Sector,WagesPerFte_9_start,WagesPerFte_9_end,GrowthMultiple
0,"A Agriculture, forestry and fishing",21.0,44.0,2.10
1,01 Agriculture,20.0,43.3,2.16
2,02 Forestry and logging,33.4,57.7,1.73
3,03 Fishing and aquaculture,46.6,70.2,1.51
4,B Mining and quarrying,53.5,104.7,1.96


In [24]:
# Task 3.2: "Growth Champion" Analysis
# ------------------------------------

# 1. Sort by 'GrowthMultiple' to find the top 10 growers
top_10_growth = df_growth.sort_values(by='GrowthMultiple', ascending=False).head(10)
growth_champion = top_10_growth.iloc[0]
print(f"🏆 Growth Champion: '{growth_champion['Sector']}' with a growth multiple of {growth_champion['GrowthMultiple']}x")

# 2. Visualize with a Bar chart
bar_chart_growth = (
    Bar()
    .add_xaxis(top_10_growth['Sector'].tolist())
    .add_yaxis("Growth Multiple", top_10_growth['GrowthMultiple'].tolist())
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(position="right"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"Top 10 Growth Sectors ({start_year}-{end_year})"),
        xaxis_opts=opts.AxisOpts(name="Growth Multiple"),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15))
    )
)
bar_chart_growth.render_notebook()


🏆 Growth Champion: '79 Travel agencies, tour operators etc' with a growth multiple of 3.92x


In [25]:
# Task 3.3: "Recession King" Analysis
# ------------------------------------

# 1. Sort by 'GrowthMultiple' ascending to find the bottom 10
bottom_10_growth = df_growth.sort_values(by='GrowthMultiple', ascending=True).head(10)
recession_king = bottom_10_growth.iloc[0]
print(f"👑 Recession King: '{recession_king['Sector']}' with a growth multiple of {recession_king['GrowthMultiple']}x")

# 2. Visualize
bar_chart_recession = (
    Bar()
    .add_xaxis(bottom_10_growth['Sector'].tolist())
    .add_yaxis("Growth Multiple", bottom_10_growth['GrowthMultiple'].tolist())
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(position="right"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"Bottom 10 Growth Sectors ({start_year}-{end_year})"),
        xaxis_opts=opts.AxisOpts(name="Growth Multiple"),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15))
    )
)
bar_chart_recession.render_notebook()


👑 Recession King: '03 Fishing and aquaculture' with a growth multiple of 1.51x


In [26]:
# Task 3.4: "Gap Multiple" Analysis
# ----------------------------------
# Finally, let's find the salary gap in the most recent year.

# 1. Find highest and lowest paying sectors in the end year
latest_year_data = df_end.copy()
highest_paying_sector = latest_year_data.loc[latest_year_data['WagesPerFte_9'].idxmax()]
lowest_paying_sector = latest_year_data.loc[latest_year_data['WagesPerFte_9'].idxmin()]

# 2. Calculate the gap multiple
gap_multiple = (highest_paying_sector['WagesPerFte_9'] / lowest_paying_sector['WagesPerFte_9']).round(2)

print(f"💰 Gap Multiple for {end_year}:")
print(f"  Highest: '{highest_paying_sector['Sector']}' at €{highest_paying_sector['WagesPerFte_9']:,}")
print(f"  Lowest:  '{lowest_paying_sector['Sector']}' at €{lowest_paying_sector['WagesPerFte_9']:,}")
print(f"  The gap multiple is {gap_multiple}x")

# 3. Visualize the gap
gap_data = pd.DataFrame([highest_paying_sector, lowest_paying_sector])
bar_chart_gap = (
    Bar()
    .add_xaxis(gap_data['Sector'].tolist())
    .add_yaxis(f"Salary in {end_year}", gap_data['WagesPerFte_9'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"Salary Gap in {end_year}"),
        yaxis_opts=opts.AxisOpts(name="Salary (€)"),
    )
)
bar_chart_gap.render_notebook()


💰 Gap Multiple for 2024:
  Highest: '19 Manufacture of coke and petroleum' at €105.1
  Lowest:  'I Accommodation and food serving' at €36.3
  The gap multiple is 2.9x


In [27]:
# ==============================================================================
# SPRINT 1 - MILESTONE 5: Deep Dive & Data Quality Check
# ==============================================================================
#
# Task 4.1: Drill-Down Analysis of Key Sectors
# ---------------------------------------------
# We will plot the yearly salary data for our four key sectors on one chart
# to visually inspect their trends and data completeness.

# 1. Identify our key sectors from previous analysis
key_sectors = [
    growth_champion['Sector'],
    recession_king['Sector'],
    highest_paying_sector['Sector'],
    lowest_paying_sector['Sector']
]
# Remove duplicates in case one sector plays multiple roles
key_sectors = list(set(key_sectors))

print("Performing drill-down analysis for the following sectors:")
for sector in key_sectors:
    print(f"- {sector}")

# 2. Filter the main df_analysis to get data for these sectors
drill_down_df = df_analysis[df_analysis['Sector'].isin(key_sectors)]

# 3. Create the multi-series line chart
drill_down_chart = Line().add_xaxis(xaxis_data=drill_down_df['Year'].unique().astype(str).tolist())

for sector in key_sectors:
    sector_data = drill_down_df[drill_down_df['Sector'] == sector]
    drill_down_chart.add_yaxis(
        series_name=sector,
        y_axis=sector_data['WagesPerFte_9'].tolist(),
        symbol_size=8,
    )

drill_down_chart.set_global_opts(
    title_opts=opts.TitleOpts(title="Drill-Down: Yearly Salary of Key Sectors"),
    tooltip_opts=opts.TooltipOpts(trigger="axis"),
    legend_opts=opts.LegendOpts(pos_top="8%"),
    toolbox_opts=opts.ToolboxOpts(is_show=True),
    xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
    yaxis_opts=opts.AxisOpts(name="Salary (€)"),
)

drill_down_chart.render_notebook()


Performing drill-down analysis for the following sectors:
- I Accommodation and food serving
- 79 Travel agencies, tour operators etc
- 03 Fishing and aquaculture
- 19 Manufacture of coke and petroleum


In [28]:
# Task 4.2: Data Quality Investigation
# ------------------------------------
# To address the concern about data completeness before 2010,
# we will create a heatmap of data availability across all years.

from pyecharts.charts import HeatMap

# 1. Use the df_master which contains all data, including NaNs
# Create a pivot table: Years as columns, Sectors as rows.
# The value will be 1 if data exists, 0 if it's NaN.
heatmap_data = df_master.pivot_table(
    index='Sector',
    columns='Year',
    values='WagesAndSalariesPerFte_1', # Use any numeric value column
    aggfunc=lambda x: 1 if not x.isnull().all() else 0
).fillna(0)

# 2. Prepare data for ECharts HeatMap format
# It needs a list of [x, y, value] points.
years = heatmap_data.columns.astype(str).tolist()
sectors = heatmap_data.index.tolist()
data_points = []
for i, sector in enumerate(sectors):
    for j, year in enumerate(years):
        value = heatmap_data.loc[sector, int(year)]
        data_points.append([j, i, value])


# 3. Create the HeatMap chart
heatmap_chart = (
    HeatMap()
    .add_xaxis(years)
    .add_yaxis(
        "Data Availability",
        sectors,
        data_points,
        label_opts=opts.LabelOpts(is_show=False), # Hide the '1's and '0's
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Data Completeness Heatmap"),
        visualmap_opts=opts.VisualMapOpts(
            is_show=False,  # Hide the color legend
            min_=0,
            max_=1,
            range_color=["#F0F0F0", "#4B8BBE"] # Grey for missing, Blue for available
        ),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45)),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size=8)),
        datazoom_opts=[
            opts.DataZoomOpts(is_show=True, type_="slider", range_start=0, range_end=100, orient="horizontal"),
            opts.DataZoomOpts(is_show=True, type_="slider", range_start=0, range_end=100, orient="vertical"),
        ],
    )
)
heatmap_chart.render_notebook()


KeyError: 'WagesAndSalariesPerFte_1'